In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import print_function
from util_functions import process_files

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
Using TensorFlow backend.


In [7]:
batch_size = 32
num_classes = 2
epochs = 200

# load the data
x_train, y_train, f_train = process_files(
                                    dataset='training', 
                                    features=['Mel', 'Mel_deltas'], 
                                    shape='stacked')

x_test, y_test, f_test = process_files(
                                    dataset='test', 
                                    features=['Mel', 'Mel_deltas'], 
                                    shape='stacked')

# input image dimensions
input_d = x_train.shape[1] #Depth
input_h = x_train.shape[2] #Height
input_w = x_train.shape[3] #Width

#Reshaping to feed to network
x_train = x_train.reshape(x_train.shape[0], input_h, input_w, input_d)
x_test = x_test.reshape(x_test.shape[0], input_h, input_w, input_d)
input_shape = (input_h, input_w, input_d)

#Making them floats for TF
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# iterate over different hyperparameters to finetune them
# from the results in 10 epochs we can see that the other learning rates are just too big. 
# Hence only this one will be used
learning_rate = 0.0001
dropout_rates = [0.3, 0.4]

for dr in dropout_rates:

    print('learning rate fixed at 0.0001, dropout: ' + str(dr) + '\n')        
    model = Sequential()
    model.add(Conv2D(80, kernel_size=(57, 6),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(80, kernel_size=(1, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(8, 7), strides=(8, 7)))
    model.add(Dropout(dr))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dr))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dr))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Nadam(lr=learning_rate),
                  metrics=['accuracy'])

    tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)
    
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              validation_split=0.1,
              callbacks=[tbCallBack])
              
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])        

    print('----------------------------------------------------\n')    

learning rate fixed at 0.0001, dropout: 0.3

Train on 1306 samples, validate on 146 samples
Epoch 1/200
 - 2s - loss: 1.5668 - acc: 0.6907 - val_loss: 1.0019 - val_acc: 0.7329
Epoch 2/200
 - 2s - loss: 1.2976 - acc: 0.7764 - val_loss: 0.3867 - val_acc: 0.8014
Epoch 3/200
 - 2s - loss: 1.0616 - acc: 0.7979 - val_loss: 0.4360 - val_acc: 0.7877
Epoch 4/200
 - 2s - loss: 1.0321 - acc: 0.8070 - val_loss: 1.4768 - val_acc: 0.5753
Epoch 5/200
 - 2s - loss: 0.8974 - acc: 0.8315 - val_loss: 0.8822 - val_acc: 0.6986
Epoch 6/200
 - 2s - loss: 0.8192 - acc: 0.8277 - val_loss: 0.5181 - val_acc: 0.7397
Epoch 7/200
 - 2s - loss: 0.7046 - acc: 0.8469 - val_loss: 1.0468 - val_acc: 0.5959
Epoch 8/200
 - 2s - loss: 0.6287 - acc: 0.8522 - val_loss: 1.8897 - val_acc: 0.4452
Epoch 9/200
 - 2s - loss: 0.5173 - acc: 0.8744 - val_loss: 0.5213 - val_acc: 0.7329
Epoch 10/200
 - 2s - loss: 0.4625 - acc: 0.8698 - val_loss: 1.2255 - val_acc: 0.6096
Epoch 11/200
 - 2s - loss: 0.4579 - acc: 0.8599 - val_loss: 1.6567 

Epoch 97/200
 - 2s - loss: 0.1338 - acc: 0.9686 - val_loss: 1.8550 - val_acc: 0.6918
Epoch 98/200
 - 2s - loss: 0.1306 - acc: 0.9709 - val_loss: 1.7239 - val_acc: 0.7192
Epoch 99/200
 - 2s - loss: 0.0996 - acc: 0.9778 - val_loss: 1.7771 - val_acc: 0.7192
Epoch 100/200
 - 2s - loss: 0.0953 - acc: 0.9801 - val_loss: 1.5129 - val_acc: 0.7466
Epoch 101/200
 - 2s - loss: 0.1456 - acc: 0.9732 - val_loss: 2.3588 - val_acc: 0.6301
Epoch 102/200
 - 2s - loss: 0.1428 - acc: 0.9686 - val_loss: 1.8331 - val_acc: 0.6712
Epoch 103/200
 - 2s - loss: 0.1041 - acc: 0.9763 - val_loss: 2.2722 - val_acc: 0.6438
Epoch 104/200
 - 2s - loss: 0.0866 - acc: 0.9770 - val_loss: 0.9824 - val_acc: 0.7671
Epoch 105/200
 - 2s - loss: 0.0819 - acc: 0.9816 - val_loss: 1.6466 - val_acc: 0.7192
Epoch 106/200
 - 2s - loss: 0.0721 - acc: 0.9832 - val_loss: 1.8771 - val_acc: 0.6918
Epoch 107/200
 - 2s - loss: 0.0747 - acc: 0.9839 - val_loss: 1.6998 - val_acc: 0.7192
Epoch 108/200
 - 2s - loss: 0.1007 - acc: 0.9770 - val_lo

Epoch 193/200
 - 2s - loss: 0.0424 - acc: 0.9877 - val_loss: 2.5468 - val_acc: 0.6644
Epoch 194/200
 - 2s - loss: 0.0413 - acc: 0.9885 - val_loss: 2.1460 - val_acc: 0.6986
Epoch 195/200
 - 2s - loss: 0.0402 - acc: 0.9893 - val_loss: 2.0498 - val_acc: 0.7329
Epoch 196/200
 - 2s - loss: 0.0423 - acc: 0.9877 - val_loss: 1.9441 - val_acc: 0.7466
Epoch 197/200
 - 2s - loss: 0.0391 - acc: 0.9885 - val_loss: 1.9269 - val_acc: 0.7466
Epoch 198/200
 - 2s - loss: 0.0394 - acc: 0.9877 - val_loss: 2.1214 - val_acc: 0.7123
Epoch 199/200
 - 2s - loss: 0.0393 - acc: 0.9877 - val_loss: 1.9762 - val_acc: 0.7397
Epoch 200/200
 - 2s - loss: 0.0391 - acc: 0.9900 - val_loss: 1.9449 - val_acc: 0.7397
Test loss: 1.1154727613782605
Test accuracy: 0.8769497400346621
----------------------------------------------------

learning rate fixed at 0.0001, dropout: 0.4

Train on 1306 samples, validate on 146 samples
Epoch 1/200
 - 2s - loss: 1.7403 - acc: 0.6845 - val_loss: 3.0681 - val_acc: 0.3151
Epoch 2/200
 - 2s 

Epoch 87/200
 - 2s - loss: 0.0921 - acc: 0.9793 - val_loss: 2.2376 - val_acc: 0.6164
Epoch 88/200
 - 2s - loss: 0.0916 - acc: 0.9832 - val_loss: 2.0038 - val_acc: 0.6438
Epoch 89/200
 - 2s - loss: 0.0834 - acc: 0.9816 - val_loss: 1.9732 - val_acc: 0.6438
Epoch 90/200
 - 2s - loss: 0.0682 - acc: 0.9824 - val_loss: 2.2353 - val_acc: 0.6370
Epoch 91/200
 - 2s - loss: 0.0692 - acc: 0.9862 - val_loss: 1.9270 - val_acc: 0.6575
Epoch 92/200
 - 2s - loss: 0.0759 - acc: 0.9793 - val_loss: 1.7980 - val_acc: 0.6507
Epoch 93/200
 - 2s - loss: 0.0910 - acc: 0.9793 - val_loss: 2.0435 - val_acc: 0.6507
Epoch 94/200
 - 2s - loss: 0.0820 - acc: 0.9786 - val_loss: 2.3313 - val_acc: 0.6370
Epoch 95/200
 - 2s - loss: 0.0898 - acc: 0.9801 - val_loss: 2.1709 - val_acc: 0.6370
Epoch 96/200
 - 2s - loss: 0.0985 - acc: 0.9786 - val_loss: 2.5621 - val_acc: 0.6096
Epoch 97/200
 - 2s - loss: 0.1071 - acc: 0.9732 - val_loss: 2.4468 - val_acc: 0.6027
Epoch 98/200
 - 2s - loss: 0.0924 - acc: 0.9755 - val_loss: 1.954

Epoch 183/200
 - 2s - loss: 0.0513 - acc: 0.9877 - val_loss: 1.9351 - val_acc: 0.7192
Epoch 184/200
 - 2s - loss: 0.0445 - acc: 0.9877 - val_loss: 1.7949 - val_acc: 0.7260
Epoch 185/200
 - 2s - loss: 0.0461 - acc: 0.9870 - val_loss: 1.8264 - val_acc: 0.7192
Epoch 186/200
 - 2s - loss: 0.0454 - acc: 0.9877 - val_loss: 2.3741 - val_acc: 0.6712
Epoch 187/200
 - 2s - loss: 0.0444 - acc: 0.9885 - val_loss: 2.3917 - val_acc: 0.6712
Epoch 188/200
 - 2s - loss: 0.0669 - acc: 0.9847 - val_loss: 1.6711 - val_acc: 0.7466
Epoch 189/200
 - 2s - loss: 0.0584 - acc: 0.9870 - val_loss: 2.5197 - val_acc: 0.6712
Epoch 190/200
 - 2s - loss: 0.0489 - acc: 0.9847 - val_loss: 2.2994 - val_acc: 0.6781
Epoch 191/200
 - 2s - loss: 0.0507 - acc: 0.9862 - val_loss: 3.1369 - val_acc: 0.6301
Epoch 192/200
 - 2s - loss: 0.0527 - acc: 0.9855 - val_loss: 3.0843 - val_acc: 0.6438
Epoch 193/200
 - 2s - loss: 0.0659 - acc: 0.9855 - val_loss: 2.6313 - val_acc: 0.6918
Epoch 194/200
 - 2s - loss: 0.0859 - acc: 0.9816 - val